# 1. 데이터가져오기

In [2]:
import pandas as pd
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:
file_path = "~/data/ChatbotData .csv"
file_path = os.path.expanduser(file_path)
data = pd.read_csv(file_path)
data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [4]:
data = data[['Q','A']]
data.head()

,Q,A
0,12시 땡!,하루가 또 가네요.
1,1지망 학교 떨어졌어,위로해 드립니다.
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.
4,PPL 심하네,눈살이 찌푸려지죠.


In [5]:
data.shape

(11823, 2)

# 2. 데이터 전처리

In [6]:
#null 체크
print(data.isnull().sum())

Q    0
A    0
dtype: int64


In [7]:
sen = "나에 대한 관심이 줄어든 거 같아,연인 사이로 발전할 수 있었던 계기를 살펴보면 서로에 대한 관심이 있었기 때문이라는 걸 잊지 마세요."

# 전처리 함수
def preprocess_sentence(sentence):
  # 입력받은 sentence를 소문자로 변경하고 양쪽 공백을 제거
  sentence = sentence.lower() #한국어에 왜 쓰는거지?

  # 특수 문자 제거
  sentence = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z?.!,]+", " ", sentence)
    
  # 여러 개의 공백을 하나로 합침
  sentence = re.sub(r'[" "]+', " ", sentence)

  sentence = sentence.strip()
  return sentence

print(preprocess_sentence(sen))

나에 대한 관심이 줄어든 거 같아,연인 사이로 발전할 수 있었던 계기를 살펴보면 서로에 대한 관심이 있었기 때문이라는 걸 잊지 마세요.


In [8]:
### Q, A 데이터 전처리
questions = []
answers = []
for [i, j] in zip(data['Q'], data['A']):
    questions.append(preprocess_sentence(i))
    answers.append(preprocess_sentence(j))

In [9]:
print(len(questions))
print(len(answers))

11823
11823


# 3. 병렬 데이터 전처리하기

## 3-1) 단어장(Vocabulary) 만들기 

In [10]:
import tensorflow_datasets as tfds

# 질문과 답변 데이터셋에 대해서 Vocabulary 생성
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(questions + answers, target_vocab_size=2**13)

In [11]:
# 전체 vocabulary 리스트 얻기
vocab_list = tokenizer.subwords 

# vocabulary 리스트 출력 
print(vocab_list[:100])  # 처음 100개 토큰만 출력
print(tokenizer.vocab_size)
print(len(vocab_list))

['거예요', '수_', '게_', '너무_', '더_', '거_', '좋아하는_', '는_', '이_', '을_', '잘_', '도_', '. ', '고_', '요', '것_', '많이_', '안_', '좋은_', '같아요', '한_', '좀_', '있어요', '싶어', '가_', '나_', '있을_', '에_', '지_', '해보세요', '은_', '사람_', '할_', '같아', '해', '네', '면_', '건_', '사람이_', '를_', '마세요', '다_', '하고_', '지', '하는_', '보세요', '죠', '어', '서_', '내가_', '의_', '다', '내_', '이제_', '마음이_', '나', '다른_', '썸_', '만_', '그_', '어떻게_', '있는_', '왜_', '싶다', '세요', '다시_', '시간이_', '수도_', '없어', '것도_', '또_', '좋을_', '오늘_', '정말_', '가', '이', '같이_', '네요', '될_', '해요', '자꾸_', '걸_', '있어', '하세요', '없어요', '일_', '제가_', '길_', '바랄게요', '로_', '까', '돼요', '하면_', '봐요', '할까', '때_', '저도_', '으로_', '먼저_', '있을까']
8168
7911


In [12]:
# 시작 토큰과 종료 토큰에 고유한 정수를 부여합니다.
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

print('START_TOKEN의 번호 :' ,[tokenizer.vocab_size])
print('END_TOKEN의 번호 :' ,[tokenizer.vocab_size + 1])

START_TOKEN의 번호 : [8168]
END_TOKEN의 번호 : [8169]


In [13]:
VOCAB_SIZE = tokenizer.vocab_size + 2
print(VOCAB_SIZE)

8170


## 3-2) 각 단어를 고유한 정수로 인코딩(Integer encoding) & 패딩(Padding)

In [14]:
print('정수 인코딩 후의 21번째 질문 샘플: {}'.format(tokenizer.encode(questions[21])))
print('정수 인코딩 후의 21번째 답변 샘플: {}'.format(tokenizer.encode(answers[21])))

정수 인코딩 후의 21번째 질문 샘플: [5756, 607, 2488, 4159]
정수 인코딩 후의 21번째 답변 샘플: [2354, 7507, 5, 6270, 94, 7958]


In [15]:
MAX_LENGTH = 40

In [16]:
#패딩추가 패키지
from tensorflow.keras.preprocessing.sequence import pad_sequences

#정수 인코딩
#최대 길이 초과 샘플 제거 및 패딩
def tokenize_and_filter(inputs, outputs):
    #토큰화 input, out 담을 그릇
    tokenized_inputs, tokenized_outputs = [], []
    
    for (sen1, sen2) in zip(inputs, outputs):
    #정수 인코딩 및 시작 토큰과 종료 토큰 추가
        sen1 = START_TOKEN + tokenizer.encode(sen1) + END_TOKEN
        sen2 = START_TOKEN + tokenizer.encode(sen2) + END_TOKEN
        
        #문장 길이가 40 이상인 문장만 추가
        if len(sen1) <= MAX_LENGTH and len(sen2) <= MAX_LENGTH:
            tokenized_inputs.append(sen1)
            tokenized_outputs.append(sen2)
    
    tokenized_inputs = pad_sequences(
        tokenized_inputs,
        maxlen = MAX_LENGTH,
        padding = "post", #시퀀스의 끝 부분에 패딩 추가
    ) 
    
    tokenized_outputs = pad_sequences(
        tokenized_outputs,
        maxlen = MAX_LENGTH,
        padding = "post", #시퀀스의 끝 부분에 패딩 추가
    ) 
    
    return tokenized_inputs, tokenized_outputs 

In [17]:
questions, answers = tokenize_and_filter(questions, answers)
print('단어장의 크기 :',(VOCAB_SIZE))
print('필터링 후의 질문 샘플 개수: {}'.format(len(questions)))
print('필터링 후의 답변 샘플 개수: {}'.format(len(answers)))

단어장의 크기 : 8170
필터링 후의 질문 샘플 개수: 11823
필터링 후의 답변 샘플 개수: 11823


## 3-3) 교사 강요(Teacher Forcing) 사용하기

In [18]:
#tf.data.Dataset API의 입력으로 사용하여 파이프라인을 구성

BATCH_SIZE = 64
BUFFER_SIZE = 20000


#from_tensor.slices : numpy나 텐서 slice 후 객체생성
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs' : questions, #인코더의 입력 
        'dec_inputs' : answers[:,:-1] 
        #답변의 마지막 토큰을 제외한 부분을 디코더 입력으로 사용
    },
    {
         'outputs' : answers[:,1:]
        #답변 데이터에서 첫 번째 토큰을 제외한 부분을 디코더의 레이블(정답)로 사용
    },

))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

# 4. 모델 정의 

## 4-0) PositionalEncoding

In [19]:
# 포지셔널 인코딩 레이어
class PositionalEncoding(tf.keras.layers.Layer):

  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    # 각도 배열 생성
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)

    # 배열의 짝수 인덱스에는 sin 함수 적용
    sines = tf.math.sin(angle_rads[:, 0::2])
    # 배열의 홀수 인덱스에는 cosine 함수 적용
    cosines = tf.math.cos(angle_rads[:, 1::2])

    # sin과 cosine이 교차되도록 재배열
    pos_encoding = tf.stack([sines, cosines], axis=0)
    pos_encoding = tf.transpose(pos_encoding,[1, 2, 0]) 
    pos_encoding = tf.reshape(pos_encoding, [position, d_model])

    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

## 4-1) scaled_dot_product_attention

In [35]:
# 스케일드 닷 프로덕트 어텐션 함수
def scaled_dot_product_attention(query, key, value, mask):
    #Query와 K의 점곱
    matmul_qk = tf.matmul(query, key, transpose_b = True)
    
    #임베딩 차원을 가져와 실수 
    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth) #임베딩차원의 제곱근으로 나눠줌
    
    #------------------------------------------------------
    # 패딩에 마스크 추가
    # 패딩인 부분은 점수계산에 팜여하지 안도록 함
    if mask is not None:
        logits += (mask * -1e9)
    #------------------------------------------------------
    
    #soft max
    attention_weights = tf.nn.softmax(logits, axis=-1)
    
    # 최종 어텐션 V의 닷 프로덕트
    output = tf.matmul(attention_weights, value)
    
    return output

## 4-2 ) MultiHeadAttention

In [32]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__ (self, d_model, num_heads, name="multi_head_attention"):
        #부모 클래스 생성자 호출, 이름을 name으로 설정
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads #나누는 헤드 개수
        self.d_model = d_model #임베딩 차원
        
        #임베딩 차원이 헤드와 나누어 떨어지는지 확인
        #만족하지 않을 시 에러
        assert d_model % self.num_heads == 0
        
        #각 헤드의 차원 계산
        self.depth = d_model // self.num_heads
        
        #각각 쿼리(Query), 키(Key), 값(Value) 벡터를 생성하기 위한 Dense 레이어
        #가중치 행렬을 곱하고 편향을 더하여 출력 벡터를 생성하는 선형 변환 레이어
        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)
        
        #어텐션 연산 후 최종 결과를 생성하기 위한 Dense 레이어
        self.dense = tf.keras.layers.Dense(units=d_model)
        
    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(
            inputs,
            shape = (batch_size,-1,self.num_heads,self.depth)
        )
        return tf.transpose(inputs, perm=[0, 2, 1, 3])
    
    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs[
            'value'], inputs['mask']
        batch_size = tf.shape(query)[0]

         # Q, K, V에 각각 Dense를 적용합니다.
        query = self.query_dense(query)  # (batch_size, seq_len, d_model)
        key = self.key_dense(key)        # (batch_size, seq_len, d_model)
        value = self.value_dense(value)  # (batch_size, seq_len, d_model)

        # 병렬 연산을 위한 머리를 여러 개 만듭니다.
        query = self.split_heads(query, batch_size)  # (batch_size, num_heads, seq_len, depth)
        key = self.split_heads(key, batch_size)      # (batch_size, num_heads, seq_len, depth)
        value = self.split_heads(value, batch_size)  # (batch_size, num_heads, seq_len, depth)

        # 스케일드 닷 프로덕트 어텐션 함수
        scaled_attention = scaled_dot_product_attention(query, key, value, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        # 어텐션 연산 후에 각 결과를 다시 연결(concatenate)합니다
        concat_attention = tf.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))

        # 최종 결과에도 Dense를 한 번 더 적용합니다
        outputs = self.dense(concat_attention)

        return outputs

## 4-3) Masking

In [22]:
#숫자가 0인 부분 체크
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, sequence length)
  return mask[:, tf.newaxis, tf.newaxis, :]

In [23]:
#다음단어 가리기
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x)
  return tf.maximum(look_ahead_mask, padding_mask)

## 4-4) 인코더

### 인코더 하나의 레이어

In [24]:
# 인코더 하나의 레이어를 함수로 구현.
# 이 하나의 레이어 안에는 두 개의 서브 레이어가 존재합니다.
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

  # 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })

  # 어텐션의 결과는 Dropout과 Layer Normalization이라는 훈련을 돕는 테크닉을 수행
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  # 두 번째 서브 레이어 : 2개의 완전연결층
  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 완전연결층의 결과는 Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

### 인코더 층

In [25]:
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 임베딩 레이어
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

  # 포지셔널 인코딩
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # num_layers만큼 쌓아올린 인코더의 층.
  for i in range(num_layers):
    outputs = encoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

## 4-5) 디코더

### 디코더 레이어

In [26]:
# 디코더 하나의 레이어를 함수로 구현.
# 이 하나의 레이어 안에는 세 개의 서브 레이어가 존재합니다.
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })

  # 멀티 헤드 어텐션의 결과는 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

  # 두 번째 서브 레이어 : 마스크드 멀티 헤드 어텐션 수행 (인코더-디코더 어텐션)
  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })

  # 마스크드 멀티 헤드 어텐션의 결과는
  # Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

  # 세 번째 서브 레이어 : 2개의 완전연결층
  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 완전연결층의 결과는 Dropout과 LayerNormalization 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

### 디코더 층

In [27]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')

  # 패딩 마스크
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')
  
  # 임베딩 레이어
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

  # 포지셔널 인코딩
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  # Dropout이라는 훈련을 돕는 테크닉을 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = decoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

## 4-6) 트랜스포머 함수 정의

In [28]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="transformer"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  # 인코더에서 패딩을 위한 마스크
  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)

  # 디코더에서 미래의 토큰을 마스크 하기 위해서 사용합니다.
  # 내부적으로 패딩 마스크도 포함되어져 있습니다.
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)

  # 두 번째 어텐션 블록에서 인코더의 벡터들을 마스킹
  # 디코더에서 패딩을 위한 마스크
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

  # 인코더
  enc_outputs = encoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[inputs, enc_padding_mask])

  # 디코더
  dec_outputs = decoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  # 완전연결층
  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

# 5. 모델 생성 및 학습

## 5-1) 모델 생성

In [36]:
tf.keras.backend.clear_session()

# 하이퍼파라미터
NUM_LAYERS = 2 # 인코더와 디코더의 층의 개수
D_MODEL = 256 # 인코더와 디코더 내부의 입, 출력의 고정 차원
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수 
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기
DROPOUT = 0.1 # 드롭아웃의 비율

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

model.summary()

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
dec_inputs (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
enc_padding_mask (Lambda)       (None, 1, 1, None)   0           inputs[0][0]                     
__________________________________________________________________________________________________
encoder (Functional)            (None, None, 256)    3145728     inputs[0][0]                     
                                                                 enc_padding_mask[0][0] 

## 5-2) 손실함수

In [37]:
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  
  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

## 5-3) 커스텀 된 학습률(Learning rate)

In [38]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

## 5-4) 모델 컴파일

In [39]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

## 5-5) 훈련하기

In [41]:
EPOCHS = 20
model.fit(dataset, epochs=EPOCHS, verbose=1)

Epoch 1/20
185/185 [==============================] - 14s 55ms/step - loss: 1.4564 - accuracy: 0.0297
Epoch 2/20
185/185 [==============================] - 10s 53ms/step - loss: 1.1851 - accuracy: 0.0495
Epoch 3/20
185/185 [==============================] - 10s 54ms/step - loss: 1.0077 - accuracy: 0.0505
Epoch 4/20
185/185 [==============================] - 10s 54ms/step - loss: 0.9296 - accuracy: 0.0540
Epoch 5/20
185/185 [==============================] - 10s 54ms/step - loss: 0.8709 - accuracy: 0.0574
Epoch 6/20
185/185 [==============================] - 10s 54ms/step - loss: 0.8106 - accuracy: 0.0617
Epoch 7/20
185/185 [==============================] - 10s 54ms/step - loss: 0.7446 - accuracy: 0.0677
Epoch 8/20
185/185 [==============================] - 10s 54ms/step - loss: 0.6715 - accuracy: 0.0755
Epoch 9/20
185/185 [==============================] - 10s 54ms/step - loss: 0.5921 - accuracy: 0.0841
Epoch 10/20
185/185 [==============================] - 10s 54ms/step - loss: 0.510

# 6. 챗봇 테스트

In [42]:
def decoder_inference(sentence):
  sentence = preprocess_sentence(sentence)

  # 입력된 문장을 정수 인코딩 후, 시작 토큰과 종료 토큰을 앞뒤로 추가.
  # ex) Where have you been? → [[8331   86   30    5 1059    7 8332]]
  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  # 디코더의 현재까지의 예측한 출력 시퀀스가 지속적으로 저장되는 변수.
  # 처음에는 예측한 내용이 없음으로 시작 토큰만 별도 저장. ex) 8331
  output_sequence = tf.expand_dims(START_TOKEN, 0)

  # 디코더의 인퍼런스 단계
  for i in range(MAX_LENGTH):
    # 디코더는 최대 MAX_LENGTH의 길이만큼 다음 단어 예측을 반복합니다.
    predictions = model(inputs=[sentence, output_sequence], training=False)
    predictions = predictions[:, -1:, :]

    # 현재 예측한 단어의 정수
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약 현재 예측한 단어가 종료 토큰이라면 for문을 종료
    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    # 예측한 단어들은 지속적으로 output_sequence에 추가됩니다.
    # 이 output_sequence는 다시 디코더의 입력이 됩니다.
    output_sequence = tf.concat([output_sequence, predicted_id], axis=-1)

  return tf.squeeze(output_sequence, axis=0)

In [43]:
def sentence_generation(sentence):
  # 입력 문장에 대해서 디코더를 동작 시켜 예측된 정수 시퀀스를 리턴받습니다.
  prediction = decoder_inference(sentence)

  # 정수 시퀀스를 다시 텍스트 시퀀스로 변환합니다.
  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('입력 : {}'.format(sentence))
  print('출력 : {}'.format(predicted_sentence))

  return predicted_sentence

In [44]:
sentence_generation('배가 너무 고파')

입력 : 배가 너무 고파
출력 : 뭐 좀 챙겨드세요.


'뭐 좀 챙겨드세요.'

In [45]:
sentence_generation('너는 누구니?')

입력 : 너는 누구니?
출력 : 저는 위로해드리는 로봇이에요.


'저는 위로해드리는 로봇이에요.'

In [47]:
sentence_generation('너는 좋아하는 사람이 있니?')

입력 : 너는 좋아하는 사람이 있니?
출력 : 뭐라고 대답할지 고민이에요.


'뭐라고 대답할지 고민이에요.'

In [48]:
sentence_generation('제일 친한 친구가 누구야?')

입력 : 제일 친한 친구가 누구야?
출력 : 운명이자 기적이죠.


'운명이자 기적이죠.'

In [49]:
# 10 에폭 추가 학습 (총 30 에폭)
model.fit(dataset, epochs=30, initial_epoch=20, verbose=1) 

Epoch 21/30
185/185 [==============================] - 10s 54ms/step - loss: 0.0418 - accuracy: 0.1650
Epoch 22/30
185/185 [==============================] - 10s 54ms/step - loss: 0.0406 - accuracy: 0.1649
Epoch 23/30
185/185 [==============================] - 10s 54ms/step - loss: 0.0370 - accuracy: 0.1659
Epoch 24/30
185/185 [==============================] - 10s 54ms/step - loss: 0.0316 - accuracy: 0.1672
Epoch 25/30
185/185 [==============================] - 10s 54ms/step - loss: 0.0271 - accuracy: 0.1683
Epoch 26/30
185/185 [==============================] - 10s 54ms/step - loss: 0.0247 - accuracy: 0.1689
Epoch 27/30
185/185 [==============================] - 10s 54ms/step - loss: 0.0216 - accuracy: 0.1696
Epoch 28/30
185/185 [==============================] - 10s 54ms/step - loss: 0.0198 - accuracy: 0.1702
Epoch 29/30
185/185 [==============================] - 10s 54ms/step - loss: 0.0181 - accuracy: 0.1705
Epoch 30/30
185/185 [==============================] - 10s 54ms/step - lo

In [50]:
sentence_generation('배가 너무 고파')

입력 : 배가 너무 고파
출력 : 뭐 좀 챙겨드세요.


'뭐 좀 챙겨드세요.'

In [51]:
sentence_generation('너는 누구니?')

입력 : 너는 누구니?
출력 : 저는 위로봇입니다.


'저는 위로봇입니다.'

In [52]:
sentence_generation('너는 좋아하는 사람이 있니?')

입력 : 너는 좋아하는 사람이 있니?
출력 : 언젠간 누구도 비정상이라 규정지을 수 없어요.


'언젠간 누구도 비정상이라 규정지을 수 없어요.'

In [53]:
sentence_generation('제일 친한 친구가 누구야?')

입력 : 제일 친한 친구가 누구야?
출력 : 저는 생각을 덜하려고 노력해요. 다른 일에 집중하거나요.


'저는 생각을 덜하려고 노력해요. 다른 일에 집중하거나요.'

**프로젝트 회고**
1. 배운 점
   - 이번 lms를 진행하면서 transformor의 개념을  이해할 수 있었음
2. 아쉬운 점
   - 데이터량이 부족하고, 모델 볼륨이 낮아서 답변이 아쉬웠음
4. 느낀 점
   - 답변이 재밌었음
6. 어려웠던 점
   - 이전에는 레이어만 쌓은 모델만 실습하다가 복잡한 모델을 경험해서 그런지 공부가 더 필요해보임